In [ ]:
!pip install yfinance pandas numpy scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
# 1. Parameters
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'META']
WINDOW_SIZE = 20  # e.g., past 20 days
TEST_SIZE = 0.2
RANDOM_STATE = 42


In [ ]:
# 2. Download data
def download_data(tickers, start=None, end=None):
    df = yf.download(tickers, period='2y', auto_adjust=False)
    # Flatten column hierarchy
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    return df

data = download_data(TICKERS)

[*********************100%***********************]  3 of 4 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['AAPL', 'GOOGL', 'MSFT', 'META']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  3 of 4 completed

In [ ]:
print("Columns in DataFrame:", data.columns.tolist())
print("TICKERS:", TICKERS)


Columns in DataFrame: ['Adj Close_AAPL', 'Adj Close_GOOGL', 'Adj Close_META', 'Adj Close_MSFT', 'Close_AAPL', 'Close_GOOGL', 'Close_META', 'Close_MSFT', 'High_AAPL', 'High_GOOGL', 'High_META', 'High_MSFT', 'Low_AAPL', 'Low_GOOGL', 'Low_META', 'Low_MSFT', 'Open_AAPL', 'Open_GOOGL', 'Open_META', 'Open_MSFT', 'Volume_AAPL', 'Volume_GOOGL', 'Volume_META', 'Volume_MSFT']
TICKERS: ['AAPL', 'MSFT', 'GOOGL', 'META']


In [ ]:
# 3. Feature engineering
#   Compute RSI, SMA, VWAP

def compute_indicators(df):
    result = []
    for ticker in TICKERS:
        o = df[f'Open_{ticker}']
        h = df[f'High_{ticker}']
        l = df[f'Low_{ticker}']
        c = df[f'Close_{ticker}']
        v = df[f'Volume_{ticker}']
        # SMA
        sma = c.rolling(window=14).mean()
        # RSI
        delta = c.diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        # VWAP
        vwap = ( (h + l + c) / 3 * v ).cumsum() / v.cumsum()
        # assemble
        df_t = pd.DataFrame({
            'Open': o,
            'High': h,
            'Low': l,
            'Close': c,
            'Volume': v,
            'SMA_14': sma,
            'RSI_14': rsi,
            'VWAP': vwap
        })
        # Label: next-day movement
        df_t['Future_Close'] = df_t['Close'].shift(-1)
        df_t['Target'] = (df_t['Future_Close'] > df_t['Close']).astype(int)
        df_t.dropna(inplace=True)
        # Prefix ticker
        df_t.columns = [f'{ticker}_{col}' for col in df_t.columns]
        result.append(df_t)
    return pd.concat(result, axis=1).dropna()

features = compute_indicators(data)


In [ ]:
# 4. Prepare sequences
def create_sequences(df, window_size):
    X, y = [], []
    values = df.values
    for i in range(len(values) - window_size):
        X.append(values[i:i + window_size, :-2])  # all features except Future_Close & Target
        y.append(values[i + window_size - 1, -1])  # Target at end of window
    return np.array(X), np.array(y)

X, y = create_sequences(features, WINDOW_SIZE)

# 5. Train/test split & scaling
nsamples, ntimesteps, nfeatures = X.shape
X = X.reshape(-1, nfeatures)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape(nsamples, ntimesteps, nfeatures)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=False
)

# 6. Build CNN model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(WINDOW_SIZE, nfeatures)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 7. Train
epochs = 20
batch_size = 32

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size
)

# 8. Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 9. Save model
model.save('cnn_stock_classifier.h5')

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

# 1) Parameters
TICKERS     = ['AAPL','MSFT','GOOGL','META']
WINDOW_SIZE = 20
TEST_SIZE   = 0.2
RANDOM_SEED = 42

# 2) Download & flatten
raw = yf.download(TICKERS, period='2y', auto_adjust=False)
raw.columns = ['_'.join(col) for col in raw.columns]

# 3) Indicators + Target
def compute_indicators(df, window=14):
    blocks = []
    for t in TICKERS:
        o = df[f'{t}_Open'];
        h = df[f'{t}_High']
        l = df[f'{t}_Low'];
        c = df[f'{t}_Close']
        v = df[f'{t}_Volume']
        sma = c.rolling(window).mean()
        delta = c.diff()
        gain  = delta.clip(lower=0).rolling(window).mean()
        loss  = -delta.clip(upper=0).rolling(window).mean()
        rs    = gain / (loss + 1e-8)
        rsi   = 100 - (100/(1+rs))
        tp    = (h + l + c)/3
        vwap  = (tp * v).cumsum() / (v.cumsum() + 1e-8)

        df_t = pd.DataFrame({
            'Open': o, 'High': h, 'Low': l, 'Close': c, 'Volume': v,
            'SMA': sma, 'RSI': rsi, 'VWAP': vwap
        })
        df_t['Target'] = (df_t['Close'].shift(-1) > df_t['Close']).astype(int)
        df_t.dropna(inplace=True)
        df_t.columns = [f'{t}_{col}' for col in df_t.columns]
        blocks.append(df_t)

    feat = pd.concat(blocks, axis=1).dropna()
    # drop the shifted Future_Close columns if you created them; here we only have one 'Target'
    return feat

features = compute_indicators(raw)

# 4) Sequence creation
def create_sequences(df, window_size):
    X, y = [], []
    vals = df.values
    n = len(vals) - window_size
    for i in range(n):
        X.append(vals[i : i + window_size, :-1])
        y.append(vals[i + window_size,  -1])
    return np.array(X), np.array(y)

X, y = create_sequences(features, WINDOW_SIZE)

# 5) Scale & split
n_s, n_t, n_f = X.shape
X_flat = X.reshape(-1, n_f)
scaler = StandardScaler().fit(X_flat)
X_scaled = scaler.transform(X_flat).reshape(n_s, n_t, n_f)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=TEST_SIZE, shuffle=False, random_state=RANDOM_SEED
)

# 6) Build & compile CNN
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(WINDOW_SIZE, n_f)),
    BatchNormalization(), MaxPooling1D(2), Dropout(0.2),
    Conv1D(64, 3, activation='relu'), BatchNormalization(),
    MaxPooling1D(2), Dropout(0.2),
    Flatten(), Dense(64, activation='relu'), Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model.summary()

# 7) Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20, batch_size=32
)

# 8) Eval & save
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Acc: {acc:.4f}")
model.save('cnn_stock_classifier.h5')


[*********************100%***********************]  4 of 4 completed
ERROR:yfinance:
4 Failed downloads:
ERROR:yfinance:['GOOGL', 'AAPL', 'META', 'MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyError: 'AAPL_Open'

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# Part A: Tokenization and Stemming using NLTK

def tokenize_and_stem(text):
    """
    Tokenizes the input text and returns a list of stemmed tokens.
    """
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# Part B: Lemmatization and Stopwords Removal using NLTK

def lemmatize_and_remove_stopwords_nltk(text):
    """
    Tokenizes the text, removes English stopwords, and returns a list of lemmas.
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    return [lemmatizer.lemmatize(token.lower())
            for token in tokens
            if token.isalpha() and token.lower() not in stop_words]

# Part C: Lemmatization and Stopwords Removal using spaCy

nlp = spacy.load('en_core_web_sm')

def lemmatize_and_remove_stopwords_spacy(text):
    """
    Processes the text with spaCy, removes stopwords and punctuation,
    and returns a list of token lemmas.
    """
    doc = nlp(text)
    return [token.lemma_ for token in doc
            if not token.is_stop and not token.is_punct]

# Part D: Bag-of-Words using CountVectorizer

def bag_of_words_count(texts, max_features=None):
    """
    Fits a CountVectorizer on the list of texts and returns
    the feature matrix and vectorizer instance.
    """
    vectorizer = CountVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Part E: Bag-of-n-grams using CountVectorizer

def bag_of_ngrams_count(texts, n_range=(1, 2), max_features=None):
    """
    Fits a CountVectorizer that captures n-grams in the specified range
    and returns the feature matrix and vectorizer instance.
    """
    vectorizer = CountVectorizer(ngram_range=n_range, max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Part F: TF-IDF Bag-of-Words using TfidfVectorizer

def bag_of_words_tfidf(texts, max_features=None):
    """
    Fits a TfidfVectorizer on the list of texts and returns
    the TF-IDF feature matrix and vectorizer instance.
    """
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Example usage
def main():
    sample_texts = [
        "Natural Language Processing (NLP) is fascinating.",
        "Tokenization, stemming, and lemmatization are crucial steps."
    ]

    # Preprocessing examples
    print("[NLTK] Tokenize & Stem:", tokenize_and_stem(sample_texts[0]))
    print("[NLTK] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_nltk(sample_texts[1]))
    print("[spaCy] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_spacy(sample_texts[1]))

    # Vectorization examples
    X_bow, bow_vec = bag_of_words_count(sample_texts)
    print("\nBag-of-Words feature names:", bow_vec.get_feature_names_out())

    X_ngrams, ngram_vec = bag_of_ngrams_count(sample_texts, n_range=(1, 3))
    print("\nBag-of-n-grams (1-3) feature names:", ngram_vec.get_feature_names_out())

    X_tfidf, tfidf_vec = bag_of_words_tfidf(sample_texts)
    print("\nTF-IDF feature names:", tfidf_vec.get_feature_names_out())

if __name__== '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
# Download the missing 'punkt_tab' resource
nltk.download('punkt_tab')


# Part A: Tokenization and Stemming using NLTK

def tokenize_and_stem(text):
    """
    Tokenizes the input text and returns a list of stemmed tokens.
    """
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# Part B: Lemmatization and Stopwords Removal using NLTK

def lemmatize_and_remove_stopwords_nltk(text):
    """
    Tokenizes the text, removes English stopwords, and returns a list of lemmas.
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    return [lemmatizer.lemmatize(token.lower())
            for token in tokens
            if token.isalpha() and token.lower() not in stop_words]

# Part C: Lemmatization and Stopwords Removal using spaCy

nlp = spacy.load('en_core_web_sm')

def lemmatize_and_remove_stopwords_spacy(text):
    """
    Processes the text with spaCy, removes stopwords and punctuation,
    and returns a list of token lemmas.
    """
    doc = nlp(text)
    return [token.lemma_ for token in doc
            if not token.is_stop and not token.is_punct]

# Part D: Bag-of-Words using CountVectorizer

def bag_of_words_count(texts, max_features=None):
    """
    Fits a CountVectorizer on the list of texts and returns
    the feature matrix and vectorizer instance.
    """
    vectorizer = CountVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Part E: Bag-of-n-grams using CountVectorizer

def bag_of_ngrams_count(texts, n_range=(1, 2), max_features=None):
    """
    Fits a CountVectorizer that captures n-grams in the specified range
    and returns the feature matrix and vectorizer instance.
    """
    vectorizer = CountVectorizer(ngram_range=n_range, max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Part F: TF-IDF Bag-of-Words using TfidfVectorizer

def bag_of_words_tfidf(texts, max_features=None):
    """
    Fits a TfidfVectorizer on the list of texts and returns
    the TF-IDF feature matrix and vectorizer instance.
    """
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

# Example usage
def main():
    sample_texts = [
        "Natural Language Processing (NLP) is fascinating.",
        "Tokenization, stemming, and lemmatization are crucial steps."
    ]

    # Preprocessing examples
    print("[NLTK] Tokenize & Stem:", tokenize_and_stem(sample_texts[0]))
    print("[NLTK] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_nltk(sample_texts[1]))
    print("[spaCy] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_spacy(sample_texts[1]))

    # Vectorization examples
    X_bow, bow_vec = bag_of_words_count(sample_texts)
    print("\nBag-of-Words feature names:", bow_vec.get_feature_names_out())

    X_ngrams, ngram_vec = bag_of_ngrams_count(sample_texts, n_range=(1, 3))
    print("\nBag-of-n-grams (1-3) feature names:", ngram_vec.get_feature_names_out())

    X_tfidf, tfidf_vec = bag_of_words_tfidf(sample_texts)
    print("\nTF-IDF feature names:", tfidf_vec.get_feature_names_out())

if __name__== '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[NLTK] Tokenize & Stem: ['natur', 'languag', 'process', '(', 'nlp', ')', 'is', 'fascin', '.']
[NLTK] Lemmatize & Remove Stopwords: ['tokenization', 'stemming', 'lemmatization', 'crucial', 'step']
[spaCy] Lemmatize & Remove Stopwords: ['Tokenization', 'stemming', 'lemmatization', 'crucial', 'step']

Bag-of-Words feature names: ['and' 'are' 'crucial' 'fascinating' 'is' 'language' 'lemmatization'
 'natural' 'nlp' 'processing' 'stemming' 'steps' 'tokenization']

Bag-of-n-grams (1-3) feature names: ['and' 'and lemmatization' 'and lemmatization are' 'are' 'are crucial'
 'are crucial steps' 'crucial' 'crucial steps' 'fascinating' 'is'
 'is fascinating' 'language' 'language processing'
 'language processing nlp' 'lemmatization' 'lemmatization are'
 'lemmatization are crucial' 'natural' 'natural language'
 'natural language processing' 'nlp' 'nlp is' 'nlp is fascinating'
 'processing' 'processing nlp' 'processing nlp is' 'stemming'
 'stemming and' 'stemming and lemmatization' 'steps' 'tokenizat

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import spacy

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
# Download the 'punkt_tab' data package
nltk.download('punkt_tab') # This line is added to download the missing data

# Part A: Tokenization and Stemming using NLTK

def tokenize_and_stem(text):
    """
    Tokenizes the input text and returns a list of stemmed tokens.
    """
    # Tokenize text into words
    tokens = word_tokenize(text)
    # Initialize the Porter Stemmer
    stemmer = PorterStemmer()
    # Stem each token
    stemmed = [stemmer.stem(token) for token in tokens]
    return stemmed

# Part B: Lemmatization and Stopwords Removal using NLTK

def lemmatize_and_remove_stopwords_nltk(text):
    """
    Tokenizes the text, removes English stopwords, and returns a list of lemmas.
    """
    # Tokenize text
    tokens = word_tokenize(text)
    # Initialize lemmatizer and stopwords list
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    # Lowercase, remove stopwords, and lemmatize
    lemmas = [lemmatizer.lemmatize(token.lower())
              for token in tokens
              if token.isalpha() and token.lower() not in stop_words]
    return lemmas

# Part C: Lemmatization and Stopwords Removal using spaCy

# Load the English model. Make sure to install with:
#   python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

def lemmatize_and_remove_stopwords_spacy(text):
    """
    Processes the text with spaCy, removes stopwords and punctuation,
    and returns a list of token lemmas.
    """
    doc = nlp(text)
    cleaned = [token.lemma_ for token in doc
               if not token.is_stop and not token.is_punct]
    return cleaned

# Example usage
def main():
    sample_text = "Natural Language Processing (NLP) is a fascinating field of AI!"
    print("Original Text:", sample_text)
    print("\n[NLTK] Tokenize & Stem:", tokenize_and_stem(sample_text))
    print("\n[NLTK] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_nltk(sample_text))
    print("\n[spaCy] Lemmatize & Remove Stopwords:", lemmatize_and_remove_stopwords_spacy(sample_text))

if __name__ == '__main__':
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Original Text: Natural Language Processing (NLP) is a fascinating field of AI!

[NLTK] Tokenize & Stem: ['natur', 'languag', 'process', '(', 'nlp', ')', 'is', 'a', 'fascin', 'field', 'of', 'ai', '!']

[NLTK] Lemmatize & Remove Stopwords: ['natural', 'language', 'processing', 'nlp', 'fascinating', 'field', 'ai']

[spaCy] Lemmatize & Remove Stopwords: ['Natural', 'Language', 'Processing', 'NLP', 'fascinating', 'field', 'AI']
